<a href="https://colab.research.google.com/github/starhou/notebooks/blob/master/ML/svm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x

In [10]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/wcd

Mounted at /gdrive
/gdrive/My Drive/wcd


In [177]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
import tensorflow as tf

import os
from tensorflow.keras import layers

from tensorflow import keras

# Helper libraries
import imageio
import numpy as np
import time
import matplotlib.pyplot as plt
from IPython import display
import PIL
import glob
from scipy import signal
from sklearn.model_selection import train_test_split
print(tf.__version__)

2.6.0


In [178]:
Batch = 1000
InputLength = 400

In [179]:
class Linear(keras.layers.Layer):
    def __init__(self, units=1, input_dim=32):
        super(Linear, self).__init__()
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            initial_value=w_init(shape=(input_dim, units), dtype="float32"),
            trainable=True,
        )
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(
            initial_value=b_init(shape=(units,), dtype="float32"), trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

In [180]:
class SVM(tf.keras.Model):
  def __init__(self, InputLength=InputLength, Batch=Batch):
    super(SVM, self).__init__()
    self.linear = Linear(1, InputLength)

  def call(self, inputs):
    x = self.linear(inputs)
    return x
  def hingeLoss(self, yr, yp):
    hinge = tf.keras.losses.Hinge()
    return tf.nn.l2_loss(self.linear.w) + hinge(yr, yp)
svm = SVM()

In [35]:
noise = tf.random.normal([10,InputLength])
generateECG = svm(noise)

In [ ]:
generateECG.numpy()

In [ ]:
svm_optimizer = tf.keras.optimizers.Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

In [181]:
# 单步训练
# 注意 `tf.function` 的使用
# 该注解使函数被“编译”
@tf.function
def train_step(real_x, real_y):
  for i in range(5):
    with tf.GradientTape(persistent=True) as tape:
      # 生成器 forward 转换 X -> Y。
      # 生成器 backward 转换 Y -> X。
      pridict_y = svm(real_x, training=True)
      svm_loss = svm.hingeLoss(real_y, pridict_y)     
    gradients_of_svm = tape.gradient(svm_loss, svm.trainable_variables)
    svm_optimizer.apply_gradients(zip(gradients_of_svm, svm.trainable_variables))
  return svm_loss, real_y, pridict_y

In [182]:
def GetAccuracy(yr, yp):
  yp = np.where(yp > 0.5, 1, -1)
  correct_prediction = np.equal(yr, yp)
  return np.mean(correct_prediction)

In [183]:
# 定义训练

def train(dataset, epochs):
  for epoch in range(epochs):
    start = time.time()
    num = 0
    resNum = 0
    yr, yp = [], []
    svm_loss, accuracy = 0, 0
    for data in dataset:
      svm_loss, yr, yp = train_step(data[0],data[1])
      accuracy = GetAccuracy(yr.numpy(), yp.numpy())
      num = num+1
      # resNum = resNum + svm_res
      # accuracy = resNum / num
      # if num%100==0:
    print ('svm_loss {}, accuracy {} \n'.format(svm_loss, accuracy))
    print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

In [186]:
database = 'mitdb'
ECG = np.load('data/new/'+database+'.npy')
# feat_label = np.load('data/new/mitdb.npy')
x = ECG[:,:-1]
y = ECG[:,-1]
x = signal.resample(x,400,axis=1)

LAMBDA = 1
seed = tf.random.normal([Batch, InputLength])

In [187]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=42)

In [188]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train,y_train)).shuffle(60000).batch(Batch)

In [ ]:
train(train_dataset, 50)

In [191]:
yprict = svm(X_test)
GetAccuracy(y_test, yprict.numpy())

0.97633203125

In [ ]:
svm.linear.w

In [193]:
svm.linear.b

<tf.Variable 'Variable:0' shape=(1,) dtype=float32, numpy=array([-0.03091024], dtype=float32)>